In [1]:
import numpy as np
from matplotlib import pyplot as plt

def lno_data(data):
      new_data = []
      lines = data.splitlines()
      for line in lines:
            columns = line.split()
            if len(columns)>1:
                  if not line.startswith("#"): 
                        new_data.append(columns)

      new_data = np.array(new_data)

      lno_thresh = []
      for i in range(new_data.shape[0]):
            thresh_vir = new_data[:,0][i].split(sep=',')[1]
            thresh_vir = float(thresh_vir.strip('(),'))
            lno_thresh.append(thresh_vir)

      lno_data = np.array(new_data[:,1:],dtype="float32")

      lno_thresh = np.array(lno_thresh,dtype="float32")
      # lno_thresh[-1] = 1e-10 # last thresh = 0.0
      lno_afqmc_corr = lno_data[:,0]
      lno_afqmc_err = lno_data[:,1]
      lno_afqmc_mp2_corr = lno_data[:,2]
      lno_ccsd_corr = lno_data[:,3]

      lno_mp2_cr = lno_afqmc_mp2_corr-lno_afqmc_corr
      lno_ccsd_mp2_corr = lno_ccsd_corr+lno_mp2_cr
      
      return lno_thresh,lno_afqmc_corr,lno_afqmc_mp2_corr,lno_afqmc_err,lno_ccsd_corr,lno_ccsd_mp2_corr

In [46]:
from pyscf import gto, scf

aB = 0.529177
a = 2*aB
b = 3*aB
nH = 6
atoms = ""
for i in range(nH):
    atoms += f"H {i*a} 0 0 \n"

mol = gto.M(atom=atoms, basis="ccpvdz", verbose=4)
mf = scf.RHF(mol)
mf.kernel()

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#1882-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Mon Sep  8 14:23:57 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 6
[INPUT] num. electrons = 6
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00

-3.219792229784061

In [47]:
from ad_afqmc.lno_ccsd import lno_ccsd
import os
options = {'n_eql': 4,
           'n_prop_steps': 30,
            'n_ene_blocks': 1,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 10,
            'seed': 98,
            'walker_type': 'rhf',
            'trial': 'cisd',
            'dt':0.005,
            'ad_mode':None,
            'use_gpu': False,
            }
threshs = [1e-4]
for i,thresh in enumerate(threshs):
    lno_ccsd.run_lno_ccsd_afqmc(mf,thresh,[],options,nproc=5)
    os.system(f'mv results.out results.out{i+1}')

lo_type = boys


******** <class 'pyscf.lo.boys.Boys'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.01
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 13.509926645871  delta_f= 13.5099  |g|= 6.40813  1 KF 3 Hx
macro= 2  f(x)= 13.204514088961  delta_f= -0.305413  |g|= 5.62056  1 KF 3 Hx
macro= 3  f(x)= 12.942188910556  delta_f= -0.262325  |g|= 4.90673  1 KF 3 Hx
macro= 4  f(x)= 12.726909556944  delta_f= -0.215279  |g|= 4.1223  1 KF 3 Hx
macro= 5  f(x)= 12.562012765114  delta_f= -0.164897  |g|= 3.27767  1 KF 3 Hx
macro= 6  f(x)= 12.450098649392  delta_f= -0.111914  |g|= 2.38462  1 KF 3 Hx
macro= 7  f(x)= 12.392984837749  delta_f= -0.0571138  |g|= 1.45562  1 KF 3 Hx
macro= 8  f(x)= 12.385311342905  delta_f= -0.00767349  |g|= 0.50352  1 KF 2 Hx


In [21]:
with open(f"/home/yichi/research/software/cs_afqmc/ad_afqmc/lno_ccsd/test/hchain/frg_1.out","r") as read_file:
    for line in read_file:
        if "lno-mp2 orb_corr" in line:
            print(line.split()[2])
            e_mp2_orb_en = line.split()[2]
        # else:
        #     print("none")
        #     e_mp2_orb_en = '  None  '
        if "lno-ccsd" in line:
            e_ccsd_orb_en = line.split()[2]
        # else:
        #     e_ccsd_orb_en = '  None  '

-0.034103


In [ ]:
from pyscf import mp
mmp = mp.MP2(mf, frozen=frozen)
e_mp2 = mmp.kernel()[0]

In [25]:
import numpy as np
def frg2result(lno_thresh,nfrag,e_mp2=None):
    with open('results.out', 'w') as out_file:
        print('# frag  mp2_orb_corr  ccsd_orb_corr' \
              '  afqmc_hf_orb_en  err  afqmc_cc_orb_en  err' \
              '  norb  nelec  time',file=out_file)
        for ifrag in range(nfrag):
            with open(f"frg_{ifrag+1}.out","r") as read_file:
                for line in read_file:
                    if "lno-mp2 orb_corr" in line:
                        e_mp2_orb_en = line.split()[2]
                    if "lno-ccsd orb_corr" in line:
                        e_ccsd_orb_en = line.split()[2]
                    if "lno-ccsd-afqmc hf_orb_en" in line:
                        hf_orb_en = line.split()[2]
                        hf_orb_en_err = line.split()[4]
                    if "lno-ccsd-afqmc cc_orb_en" in line:
                        cc_orb_en = line.split()[2]
                        cc_orb_en_err = line.split()[4]
                    if "number of active orbitals" in line:
                        norb = line.split()[4]
                    if "number of active electrons" in line:
                        nelec = line.split()[4]
                    if "total run time" in line:
                        tot_time = line.split()[3]
                print(f'{ifrag+1:3d}  '
                      f'{e_mp2_orb_en}  {e_ccsd_orb_en}  '
                      f'{hf_orb_en}  {hf_orb_en_err}  '
                      f'{cc_orb_en}  {cc_orb_en_err}  '
                      f'{norb}  {nelec}  {tot_time}', file=out_file)

    data = []
    with open('results.out', 'r') as read_file:
        for line in read_file:
            if not line.startswith("#"):
                data = np.hstack((data,line.split()))
                data[data == 'None'] = '0.000000' 

    data = np.array(data.reshape(nfrag,10))
    e_mp2_orb_en = np.array(data[:,1],dtype='float32')
    e_ccsd_orb_en = np.array(data[:,2],dtype='float32')
    hf_orb_en = np.array(data[:,3],dtype='float32')
    hf_orb_en_err = np.array(data[:,4],dtype='float32')
    cc_orb_en = np.array(data[:,5],dtype='float32')
    cc_orb_en_err = np.array(data[:,6],dtype='float32')
    norb = np.array(data[:,7],dtype='int32')
    nelec = np.array(data[:,8],dtype='int32')
    tot_time = np.array(data[:,9],dtype='float32')

    e_mp2_corr = sum(e_mp2_orb_en)
    mp2_cr = e_mp2 - e_mp2_corr
    e_ccsd_corr = sum(e_ccsd_orb_en)
    afqmc_hf_corr = sum(hf_orb_en)
    afqmc_hf_corr_err = np.sqrt(sum(hf_orb_en_err**2))
    afqmc_cc_corr = sum(cc_orb_en)
    afqmc_cc_corr_err = np.sqrt(sum(cc_orb_en_err**2))
    norb_avg = np.mean(norb)
    nelec_avg = np.mean(nelec)
    norb_max = max(norb)
    nelec_max = max(nelec)
    tot_time = sum(tot_time)

    e_mp2_corr = f'{e_mp2_corr:.6f}'
    e_ccsd_corr = f'{e_ccsd_corr:.6f}'
    afqmc_hf_corr = f'{afqmc_hf_corr:.6f}'
    afqmc_hf_corr_err = f'{afqmc_hf_corr_err:.6f}'
    afqmc_cc_corr = f'{afqmc_cc_corr:.6f}'
    afqmc_cc_corr_err = f'{afqmc_cc_corr_err:.6f}'

    with open('results.out', 'a') as out_file:
        out_file.write(f'# final results \n')
        out_file.write(f'# mean-field energy: {mf.e_tot:.8f}\n')
        out_file.write(f'# lno-thresh {lno_thresh}\n')
        out_file.write(f'# e_mp2_corr: {e_mp2_corr}\n')
        out_file.write(f'# e_ccsd_corr: {e_ccsd_corr}\n')
        out_file.write(f'# afqmc/hf_corr: {afqmc_hf_corr} +/- {afqmc_hf_corr_err}\n')
        out_file.write(f'# afqmc/cc_corr: {afqmc_cc_corr} +/- {afqmc_cc_corr_err}\n')
        out_file.write(f'# mp2_correction: {mp2_cr:.8f}\n')
        out_file.write(f'# number of orbitals: average {norb_avg:.2f} maxium {norb_max}\n')
        out_file.write(f'# number of electrons: average {nelec_avg:.2f} maxium {nelec_max}\n')
        out_file.write(f'# total run time: {tot_time:.2f}\n')
    
    return None

In [40]:
frg2result((1e-3,1e-4),3,-0.10315675)

In [ ]:
# # frag  mp2_orb_corr  ccsd_orb_corr  afqmc_hf_orb_en  err  afqmc_cc_orb_en  err  norb  nelec  time
#   1  -0.034103  -0.045305  -0.044613  0.001516  -0.045329  0.000417  14  6  51.38
#   2  -0.031766  -0.043220  -0.042663  0.002022  -0.043089  0.000081  12  4  37.65
#   3  -0.031766  -0.043220  -0.043942  0.002305  -0.042940  0.000087  12  4  37.69
# # final results 
# # mean-field energy: -3.21979223
# # lno-thresh 0.0001
# # e_mp2_corr: -0.097635
# # e_ccsd_corr: -0.131745
# # afqmc/hf_corr: -0.131218 +/- 0.003420
# # afqmc/cc_corr: -0.131358 +/- 0.000434
# # mp2_correction: -0.00552175
# # number of orbitals: average 12.67 maxium 14
# # number of electrons: average 4.67 maxium 6
# # total run time: 126.72

In [44]:
def sum_results(n_results):
    with open('sum_results.out', 'w') as out_file:
        print("# thresh(occ,vir) "
              "  mp2_corr  ccsd_corr"
              "  afqmc/hf_corr   err "
              "  afqmc/ccsd_corr   err "
              "  nelec_avg   nelec_max  "
              "  norb_avg   norb_max  "
              "  run_time",file=out_file)
        for i in range(n_results):
            with open(f"results.out{i+1}","r") as read_file:
                for line in read_file:
                    if "lno-thresh" in line:
                        thresh_occ = line.split()[-2]
                        thresh_vir = line.split()[-1]
                        thresh_occ = float(thresh_occ.strip('()[],'))
                        thresh_vir = float(thresh_vir.strip('()[],'))
                    if "e_mp2_corr:" in line:
                        mp2_corr = line.split()[-1]
                    if "e_ccsd_corr:" in line:
                        ccsd_corr = line.split()[-1]
                    if "afqmc/hf_corr:" in line:
                        afqmc_hf_corr = line.split()[-3]
                        afqmc_hf_corr_err = line.split()[-1]
                    if "afqmc/cc_corr:" in line:
                        afqmc_cc_corr = line.split()[-3]
                        afqmc_cc_corr_err = line.split()[-1]
                    if "mp2_correction:" in line:
                        mp2_cr = line.split()[-1]
                    if "electrons:" in line:
                        nelec_avg = line.split()[-3]
                        nelec_max = line.split()[-1]
                    if "orbitals:" in line:
                        norb_avg = line.split()[-3]
                        norb_max = line.split()[-1]
                    if "time:" in line:
                        run_time = line.split()[-1]
            print(f" ({thresh_occ:.2e},{thresh_vir:.2e}) \t"
                  f" {mp2_corr} \t {ccsd_corr} \t"
                  f" {afqmc_hf_corr} \t {afqmc_hf_corr_err} \t"
                  f" {afqmc_cc_corr} \t {afqmc_cc_corr_err} \t"
                  f" {mp2_cr}  {nelec_avg} \t {nelec_max} \t"
                  f" {norb_avg}  \t {norb_max} \t {run_time}",file=out_file)
    return None

In [45]:
sum_results(1)